In [1]:
import geopandas as gpd
import pandas as pd
import ee
import geemap
from geowrangler import grids
import matplotlib.pyplot as plt
from shapely.geometry import mapping

In [2]:
ee.Authenticate()

Enter verification code:  4/1AQlEd8yYvUZlMREwZeTDLsdynswhll6nO1-2yDWnR-SbXSvk-_kbW_s9mdk



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
geojson_path = 'Untitled_layer.geojson'
gdf = gpd.read_file(geojson_path)

In [5]:
grid_generator = grids.SquareGridGenerator(1_000)
grid_gdf = grid_generator.generate_grid(gdf)

In [6]:
Map = geemap.Map()
Map.add_gdf(gdf, layer_name='GeoJSON Layer', style={'color': 'red', 'fillOpacity': 0})
Map.add_gdf(grid_gdf, layer_name='1km x 1km Grid', style={'color': 'black', 'fillOpacity': 0.2})
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [7]:
geojson = gdf.__geo_interface__

In [8]:
roi = ee.Geometry(geojson['features'][0]['geometry'])
modis_aod = ee.ImageCollection("MODIS/061/MCD19A2_GRANULES")\
            .select('Optical_Depth_047') \
            .filterDate('2023-01-01', '2023-12-31')
start_date = '2023-11-01'
end_date = '2023-11-11'
modisaod_filtered = modis_aod.filterDate(start_date, end_date).filterBounds(roi)

In [9]:
def extract_values(image):
    date = ee.Date(image.get('system:time_start'))
    date_str = date.format('YYYY-MM-dd')
    time_str = date.format('HH:mm:ss')
    aod = modisaod_filtered.filterDate(date, date.advance(1, 'day')).mean().reduceRegion(
        reducer=ee.Reducer.mean(), geometry=roi, scale=1000).get('Optical_Depth_047')
    
    return ee.Feature(None, {
        'date': date_str,
        'time': time_str,
        'modis_aod': aod
    })

features = modisaod_filtered.map(extract_values).getInfo()

In [10]:
values = []
for feature in features['features']:
    date = feature['properties']['date']
    time = feature['properties']['time']
    
    aod = feature['properties'].get('modis_aod', None)
    
    values.append({
        'date': date,
        'time': time,
        'coordinates': geojson['features'][0]['geometry']['coordinates'],
        'modis_aod': aod
    })

df = pd.DataFrame(values)
df.head(75)

,date,time,coordinates,modis_aod
0,2023-11-01,00:00:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",409.133085
1,2023-11-01,01:35:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",409.133085
2,2023-11-01,03:15:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",409.133085
3,2023-11-01,04:55:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",463.792978
4,2023-11-01,06:30:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",486.740112
...,...,...,...,...
70,2023-11-01,06:35:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",486.740112
71,2023-11-01,08:10:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",573.547664
72,2023-11-01,09:50:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",573.547664
73,2023-11-01,11:30:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",573.547664


In [11]:
df.to_csv('MODIS_AOD_DATA.csv')